In this notebook two networks are compared as a way of testing how the learning is going. In particular this will show which layers are active learning and which are not.

In [ ]:
import os
import sys
import pprint

import numpy as np
import matplotlib
matplotlib.use('Agg')
%matplotlib inline
from matplotlib import pyplot as plt

In [ ]:
def add_path(path):
    if path not in sys.path:
        sys.path.insert(0, path)

# need to point the correct location of the py-faster-rcnn version of the caffe library
add_path('/home/ubuntu/src/py-faster-rcnn/caffe-fast-rcnn/python')
add_path('/home/ubuntu/src/py-faster-rcnn/lib')
import caffe
from datasets.factory import get_imdb, list_imdbs
from fast_rcnn.test import test_net
from fast_rcnn.train import get_training_roidb, train_net
from fast_rcnn.config import cfg, cfg_from_file, cfg_from_list, get_output_dir
print "Loaded caffe version {:s} from {:s}.".format(caffe.__version__, caffe.__path__[0])

In [ ]:
test_file = '/home/ubuntu/dstl/models/ZF/test.prototxt'
pretrained_file = '/home/ubuntu/src/py-faster-rcnn/data/imagenet_models/ZF.v2.caffemodel'
trained_file = '/home/ubuntu/src/py-faster-rcnn/output/faster_rcnn_end2end/dstl_train/zf_faster_rcnn_transfer_no_rpn_iter_100.caffemodel'
trained2_file = '/home/ubuntu/src/py-faster-rcnn/output/faster_rcnn_end2end/dstl_train/zf_faster_rcnn_transfer_no_rpn_iter_200.caffemodel'

def load_net(test_file, weights_file):
#     np.random.seed(cfg.RNG_SEED)
#     caffe.set_random_seed(cfg.RNG_SEED)
    net = caffe.Net(test_file, weights_file, caffe.TEST)
    return net

pretrained_net = load_net(test_file, pretrained_file)
pretrained_net_redo = load_net(test_file, pretrained_file)
trained_net = load_net(test_file, trained_file)
trained_net_redo = load_net(test_file, trained_file)
trained2_net = load_net(test_file, trained2_file)

In [ ]:
diff_layers = []
for layer, layer_name in zip(pretrained_net.layers, pretrained_net._layer_names):
    try:
        pretrained_weights = pretrained_net.params[layer_name][0].data
        trained_weights = trained_net.params[layer_name][0].data
        diff0 = abs(pretrained_weights - trained_weights).sum()
        pretrained_biases = pretrained_net.params[layer_name][1].data
        trained_biases = trained_net.params[layer_name][1].data
        diff1 = abs(pretrained_biases - trained_biases).sum()
        if diff0 + diff1:
            print "Difference for layer {:s} of type {:s}:\n{:f}, {:f}".format(layer_name, layer.type, diff0, diff1)
            print pretrained_weights.sum()
            diff_layers.append(layer_name)
    except KeyError:
#         print "Layers {:s} does not have data".format(layer_name)
        pass
print diff_layers

In [ ]:
layer_name = 'rpn_conv/3x3'
pretrained_weights = pretrained_net.params[layer_name][0].data
trained_weights = trained_net.params[layer_name][0].data
print pretrained_weights.shape, trained_weights.shape

In [ ]:
orig_test_file = '/home/ubuntu/src/py-faster-rcnn/models/pascal_voc/ZF/faster_rcnn_end2end/test.prototxt'
orig_file = '/home/ubuntu/src/py-faster-rcnn/data/imagenet_models/ZF.v2.caffemodel'
orig_net = load_net(orig_test_file, orig_file)
orig_net_redo = load_net(orig_test_file, orig_file)

In [ ]:
def show_diff(net0, net1):
    for layer_name in diff_layers:
        weights0 = net0.params[layer_name][0].data
        weights1 = net1.params[layer_name][0].data
        biases0 = net0.params[layer_name][1].data
        biases1 = net1.params[layer_name][1].data
        indices = [slice(0, I) for I in net1.params[layer_name][0].data.shape]
        weights0_trunc = weights0[indices]
        indices = [slice(0, I) for I in net1.params[layer_name][1].data.shape]
        biases0_trunc = biases0[indices]
        
        diff_weights = (weights0_trunc - weights1).std()
        diff_biases = abs(biases0_trunc - biases1).sum()
        
        size0 = net0.params[layer_name][0].data.size
        diff_weights_norm = diff_weights/np.sqrt(2) - .01
        print layer_name, diff_weights, diff_weights_norm, diff_biases, weights0.std()
        
show_diff(trained_net, trained2_net)